In [51]:
#Hadji Dejan Loaiciga Herrera
from __future__ import absolute_import,division,print_function,unicode_literals

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math

In [52]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginica']

# **Dataset**

In [53]:
train_path= tf.keras.utils.get_file("iris_trainig.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv") #Datos de entrenamiento
test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv") #Datos de prueba


In [54]:
train=pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test=pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


In [55]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Obtener las diversas especies para poder utilizarlas

In [56]:
train_y=train.pop('Species')
test_y= test.pop('Species')
train.head()
#En esta impresion no aparecerá la columna especies

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [57]:
train.shape
#Tenemos 120 entradas y 4 columnas importantes

(120, 4)

# Entrenamiento

Preparacion

In [66]:

def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((features, labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)


# **Creando el Modelo**
*   Entrenamiento




### DNN Classifier

Entrenando Red Neuronal Profunda con capas ocultas

In [81]:
my_feature_columns=[]

for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

classifier = tf.estimator.DNNClassifier(
      feature_columns=my_feature_columns,
      #Capas ocultas de 30 nodos y 10 nodos
      hidden_units=[30, 10],
      n_classes=3)

test_features = {
    "SepalLength": test["SepalLength"],
    "SepalWidth": test["SepalWidth"],
    "PetalLength": test["PetalLength"],
    "PetalWidth": test["PetalWidth"],
}

eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test_features, test_y, training=False)
)
print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy:0.267



# **Predicción**

In [93]:
def input_fn(features):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(256)
#Inicializa features
features = {
    'SepalLength': [1.1],
    'SepalWidth': [2.2],
    'PetalLength': [3.3],
    'PetalWidth': [4.4],
}

predict={}
#Ciclo for que nos solicita insertar datos

print("Ingrese los datos numericos prompt: #.#")
for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit(): valid=False

  predict[feature]=[float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(features))

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Ingrese los datos numericos prompt: #.#
SepalLength: 0.1
SepalWidth: 0.2
PetalLength: 0.1
PetalWidth: 0.1
Prediction is "Setosa" (71.7%)
